# REGRESSION

### GETTING SUMMARY STATISTICS ON COLUMNS

In [1]:
####################################################################################### INITIALIZING CURRENT_DIR ################################################################################
import os

def CURRENT_DIR():
    cwd = os.getcwd()
    return os.chdir(cwd[:(cwd.index("Eliza")+5)])

CURRENT_DIR()

In [2]:
####################################################################################### MODULE USED IN THIS NOTEBOOK ################################################################################

import pandas as pd
import seaborn as sns
import numpy as np
from typing import Optional, Union, Literal
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV 
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.impute import KNNImputer, SimpleImputer
import xgboost as xgb
from sklearn.pipeline import Pipeline
import os
from src.config import Config, ModelConfig

In [3]:
####################################################################################### GETTING TO KNOW DF_MAIN ################################################################################

# import libaries
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

#

# loading data_forsale_new.csv into df_main
df_main = ModelConfig.load_data("data/data_forsale_new.csv", "csv")

# checking out on df_main
Config.expand_display(df_main.columns)
Config.expand_display(x="correlation betwwe price and other variables", y=df_main[["price", "price_per_sqmeter", "plot_area", "habitable_surface", 
         "land_surface", "bedroom_count", "room_count", 'postalcode', 'district', 'province', 'region']].corr()["price"], )



Index(['address', 'bedroom_count', 'district', 'epc_score', 'habitable_surface', 'immo_status', 'immocode', 'land_surface', 'municipality', 'plot_area', 'postalcode', 'price', 'price_per_sqmeter', 'province', 'region', 'room_count', 'subtype', 'type'], dtype='object')

None

None

'correlation betwwe price and other variables'

price                               1.00
price_per_sqmeter                   0.17
plot_area                           0.15
habitable_surface                   0.34
land_surface                        0.15
bedroom_count                       0.38
room_count                          0.38
postalcode                         -0.12
district                           -0.22
province                            0.14
region                              0.06
Name: price, dtype: float64

None

based on the calculated correlation value above, we can pick 3 variables that have an adequately significant correlation with variable **price**:
1. **habitable_surface** : 0.344623
2. **room_count**        : 0.376763
3. **bedroom_count**     : 0.382781

But in the end, I will use these columns:
Variable            | Target/Feature   | Correlation (Poisson)
price               |                  |       1.00
price_per_sqmeter   |                  |       0.17
plot_area           |                  |       0.15
habitable_surface   |                  |       0.34
land_surface        |                  |       0.15
bedroom_count       |                  |       0.38
room_count          |                  |       0.38


# MODEL 1 - TREE BOOSTED, NONLINEAR REGRESSION POWERED BY XGBREGRESSOR

MODEL 1 is the 

In [4]:
####################################################################################### MODEL-1 ########################################################################################################

# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.config import Config, ModelConfig
from sklearn.model_selection import  train_test_split, cross_val_score, KFold
from sklearn.impute import KNNImputer
from sklearn.metrics import make_scorer

import pandas as pd
import xgboost as xgb


# Loading data_forsale_new.csv into df_main
df_set_one = ModelConfig.load_data(filepath='./data/data_forsale_new.csv', file_type="csv", usecols=["price", "plot_area", "habitable_surface", "bedroom_count", "land_surface", "room_count"])

# Apply log transformation to the 'price' column
df_set_one['price'] = np.log1p(df_set_one['price'])

# Extract the features (X) and target (y)
X, y = ModelConfig.feature_target_config(df=df_set_one)

y = y.reshape(-1, 1)

# Create preprocessing_pipeline
preprocessing_pipeline=ModelConfig.PimpMyPipeline(steps=['knn_imputer', 'poly_features', 'std_scaler'], poly_degree=5)

# fit_transform X, y using preprocessing_pipeline
X_preprocessed = preprocessing_pipeline.fit_transform(X)
y_preprocessed = preprocessing_pipeline.fit_transform(y)

# # MAKING MACHINE LEARNING MODEL
# xgb_reg, X_train, X_test, y_train, y_test, y_pred, eval_results, results = ModelConfig.XGBREGRConfig(X_preprocessed , y_preprocessed)

# train_losses = eval_results['validation_0']['mphe']
# test_losses = eval_results['validation_1']['mphe']
# epochs = range(1, len(train_losses) + 1)

# # Perform cross-validation with custom Pseudo Huber loss scorer
# cv = KFold(n_splits=7, shuffle=True, random_state=42)
# neg_mse = cross_val_score(xgb_reg, X_test, y_pred, cv=cv, scoring='neg_mean_squared_error')


# plt.figure(figsize=(10, 8))
# plt.plot(epochs, train_losses, label='Pseudo-Huber Loss During Training')
# plt.plot(epochs, test_losses, label='Pseudo-Huber Loss During Testing')
# plt.xlabel('Number of Boosting Iterations')
# plt.ylabel('Pseudo-Huber Loss')
# plt.legend()
# plt.title('Training and Validation Loss History (Pseudo-Huber Loss)')
# plt.show()

# # Scatter Plot of Predicted Values vs. Actual Values 
# actual_values = np.array(X_test)

# # # Sort the test data points based on feature values
# sort_indices = np.argsort(X_test[:, 0])
# X_test = X_test[sort_indices]
# y_test = y_test[sort_indices]
# y_pred_sorted = y_pred[sort_indices]

# # Visualize predictions
# plt.scatter(X_test[:, 0], y_test, color="darkblue", label="test data")
# plt.plot(X_test[:, 0], y_pred_sorted, color="red", label="XGBoost regression line")
# plt.title("XGBoost Regression with Polynomial Features")
# plt.xlabel('True Target Values')
# plt.ylabel('Predicted Target Values')
# plt.xscale("linear")
# plt.yscale("linear")
# plt.legend()
# plt.show()

# # Evaluation
# mphe = xgb_reg.score(X_test, y_test)
# print(f"This XGBoost Regression Model generates Mean Pseud0 Huber Error of {mphe}")




KeyError: 'eval_metric'

In [23]:
import xgboost as XGB
from sklearn.model_selection import  train_test_split, cross_val_score, KFold

xgb_reg = XGB.XGBRFRegressor()

X_train, X_test, y_train, y_test = train_test_split(
                X_preprocessed, y_preprocessed, test_size=0.20, random_state=12
            )

sort_indices = np.argsort(X_test[:, 0]) #template to begin sorting indices
X_train_sorted = X_train[sort_indices] #sorting X_train
X_test_sorted = X_test[sort_indices] #sorting x_test
y_test_sorted = y_test[sort_indices] #sorting y_test
y_train_sorted = y_train[sort_indices]# sorting y_train
# y_pred_sorted = y_pred[sort_indices]

X
print(X_train_sorted.shape, X_train.shape)

X_train_df = pd.DataFrame(X_train_sorted[:, :5], columns=["plot_area", "habitable_surface", "bedroom_count", "land_surface", "room_count"])
X_test_df = pd.DataFrame(X_test_sorted[:, :5], columns=["plot_area", "habitable_surface", "bedroom_count", "land_surface", "room_count"])
y_train_df = pd.DataFrame(y_train_sorted[:, :1], columns=["price"])
y_test_df = pd.DataFrame(y_test_sorted[:, :1], columns=["price"])

dtrain = pd.concat(objs=[X_train_df, y_train_df], axis=1)
# Config.expand_display(dtrain.head())
Config.expand_display(x=dtrain.isna().value_counts())


DTRAIN = XGB.DMatrix(data=dtrain)
Config.expand_display(x=DTRAIN.get_weight())




# Config.expand_display(X_train_df.head())




(3234, 32) (12936, 32)


plot_area  habitable_surface  bedroom_count  land_surface  room_count  price
False      False              False          False         False       False    3234
Name: count, dtype: int64

None

None

(3234, 32)


array([], dtype=float32)

None

None